Loading & Exploring the dataset

In [1]:
#Import Libraries
import numpy as np
import pandas as pd

# Load data
df = pd.read_csv("Orders_with_issues.csv")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Preview
print("🔍 Sample Data:")
print(df.head())

# Summary
print("\n🧾 Column Info:")
print(df.info())

# Check missing values
print("\n🚨 Missing Values:")
print(df.isna().sum)

# Check unique shipping companies
print("\n📦 Unique Shipping Companies:")
print(df['ShippingCompany'].value_counts())



🔍 Sample Data:
   OrderID CustomerID   OrderDate ShippedDate        ShippingCost ShipCountry  \
0   1000.0       C001  2025-05-17  2025-07-30  -57.25454602444980     Germany   
1   1001.0       C002  2025-01-26  2025-07-30              320.61      Canada   
2   1002.0       C003  2025-03-08  2025-07-30              165.17      Canada   
3   1003.0       C004  2025-03-24  2025-07-30               12.55     Germany   
4   1004.0       C005  2025-04-15  2025-07-30              186.36      Canada   

    ShipCity                 ShippingCompany  
0    Hamburg  Kiwilytics Goods Shipping LLC.  
1   Montreal                   UPS Worldwide  
2  Vancouver                 FedEx Logistics  
3     Munich            Aramex International  
4  Vancouver                 FedEx Logistics  

🧾 Column Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0  

Cleaning Dates, numeric entries

In [2]:
# Convert to Datetime Datatype
df['ShippedDate'] = pd.to_datetime(df['ShippedDate'], errors='coerce')

# Convert to Datetime Datatype
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')

# Convert to float64 Datatype
df['ShippingCost'] = pd.to_numeric(df['ShippingCost'], errors='coerce')
# Change any negative value to NaN
df.loc[df['ShippingCost'] < 0 , 'ShippingCost'] = np.nan
# Fill NaN values with Median of ShippingCost Column
df['ShippingCost'] = df['ShippingCost'].fillna(df['ShippingCost'].median())

#Drop missing rows of both dates
df = df[~(df['OrderDate'].isna() & df['ShippedDate'].isna())]

Handling Nulls, Fixing Fields

In [3]:
# forward-fill missing values
df['OrderID'] = df['OrderID'].ffill()

# Fill missing values with unknown
df['CustomerID'] = df['CustomerID'].fillna('Unknown')

# Fill missing values with unspecified
df['ShipCity'] = df['ShipCity'].fillna('Unspecified')

# Removes leading and trailing whitespace from each string & Converts the text into title case (first letter uppercase, rest lowercase)
df['ShipCountry'] = df['ShipCountry'].str.strip().str.title()
df['ShipCity'] = df['ShipCity'].str.strip().str.title()
df['ShippingCompany'] = df['ShippingCompany'].str.strip()

# finding rows where the ShippingCompany column contains the word "Kiwilytics" and replacing it
df.loc[df['ShippingCompany'].str.contains('Kiwilytics', na=False), 'ShippingCompany'] = 'Kiwilytics Goods Shipping LLC.'

Feature Engineering

In [4]:
# Days between order and shipment
df['DeliveryDays'] = df['ShippedDate'] - df['OrderDate']
# extracts the number of days from a timedelta object
df['DeliveryDays'] = df['DeliveryDays'].dt.days

def get_status(x):
    if pd.isna(x):
        return 'Unknown'
    elif x > 60:
        return "Late"
    else:
        return 'OnTime'

df['DeliveryStatus'] = df['DeliveryDays'].apply(get_status)

domestic_countries = ['Germany']
def check_domestic(country):
    if country in domestic_countries:
        return 'Yes'
    else:
        return 'No'

df['IsDomestic'] = df['ShipCountry'].apply(check_domestic)

df['AffiliationStatus'] = df['ShippingCompany'].str.contains('Kiwilytics', na=False).map({True: "Affiliated", False:"External"})

avg_shipping_by_company = df.groupby('ShippingCompany')['ShippingCost'].mean()

Export & Reporting

In [5]:
# Export cleaned file
df.to_csv("cleaned_orders_final.csv",index=False)

# Final summary
print("\n✅ Final Dataset Snapshot:")
print(df.head())

print("\n📈 Delivery Status Breakdown:")
print(df['DeliveryStatus'].value_counts())


print("\n🌎 Orders by Country:")
print(df['ShipCountry'].value_counts())

print("\n🌎 Orders by City:")
print(df['ShipCity'].value_counts())

print("\n📦 Top 3 Shipping Companies:")

print(df['ShippingCompany'].value_counts().head(3))

print("\n📦 Orders by Affiliation:")

print(df['AffiliationStatus'].value_counts())


✅ Final Dataset Snapshot:
   OrderID CustomerID  OrderDate ShippedDate  ShippingCost ShipCountry  \
0   1000.0       C001 2025-05-17  2025-07-30        234.09     Germany   
1   1001.0       C002 2025-01-26  2025-07-30        320.61      Canada   
2   1002.0       C003 2025-03-08  2025-07-30        165.17      Canada   
3   1003.0       C004 2025-03-24  2025-07-30         12.55     Germany   
4   1004.0       C005 2025-04-15  2025-07-30        186.36      Canada   

    ShipCity                 ShippingCompany  DeliveryDays DeliveryStatus  \
0    Hamburg  Kiwilytics Goods Shipping LLC.          74.0           Late   
1   Montreal                   UPS Worldwide         185.0           Late   
2  Vancouver                 FedEx Logistics         144.0           Late   
3     Munich            Aramex International         128.0           Late   
4  Vancouver                 FedEx Logistics         106.0           Late   

  IsDomestic AffiliationStatus  
0        Yes        Affiliated  